### 使用RAG搭建FAQ bot

In [19]:
import getpass
import os

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [11]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [23]:
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path = (
    "../datasets/faq.csv"
)

loader = CSVLoader(file_path=file_path, csv_args={
    'delimiter': '\t',
    'fieldnames': ['question', 'answer']
})
data = loader.load()

for record in data[:2]:
    print(record)

RuntimeError: Error loading ../datasets/faq.csv

### 创建splits并搭建vectorstore，包含question和answer

In [12]:
vectorstore = Chroma.from_documents(documents=data, embedding=OpenAIEmbeddings())

In [17]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

retrieved_docs = retriever.invoke("Can I order without creating an account?")

In [18]:
retrieved_docs

[Document(page_content='question: Can I order without creating an account?\nanswer: Yes, you can place an order as a guest without creating an account. However, creating an account offers benefits such as order tracking and easier future purchases.', metadata={'row': 17, 'source': '../datasets/faq.csv'}),
 Document(page_content='question: Can I order by phone?\nanswer: Unfortunately, we do not accept orders over the phone. Please place your order through our website for a smooth and secure transaction.', metadata={'row': 13, 'source': '../datasets/faq.csv'})]

In [21]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [22]:
response = rag_chain.invoke({"input": "How do returns work"})
print(response["answer"])

You can raise a return request by logging into your Flipkart account, going to My Orders, clicking on 'Return' for the item, and filling in the details. Once the request is raised, you'll receive confirmation through email and SMS. Depending on the item, the request may be automatically approved or you may be contacted for more details before the item is picked up for replacement or refund.
